### TODO

- Correct it, it's finding local mimima
- Find the right way to turn the solution into integers

In [135]:
import numpy as np
import scipy.optimize as optimize
from random import randint
import sys

%xmode plain

def init_simulation(num_nodes, 
                    num_days, 
                    icu_capacities = None, 
                    transport_capacities = None, 
                    ini_path = None, 
                    demand_min = 80, 
                    demand_max = 90,
                    icu_min = 10,
                    icu_max = 300,
                    transport_min = 20,
                    transport_max = 30):
    if ini_path:
        print("you can also upload an .ini file")
        raise NotImplemented
    else:
        demand = []
        for node in range(num_nodes):
            demand.append(list(np.random.randint(demand_min, demand_max, size = num_days)))
        
        icu_capacities = icu_capacities if icu_capacities else list(np.random.randint(icu_min, 
                                                                                      icu_max, 
                                                                                      size = num_nodes))
        transport_capacities = transport_capacities if transport_capacities else list(np.random.randint(transport_min, 
                                                                                                        transport_max, 
                                                                                                        size = num_nodes)) 
    return icu_capacities, transport_capacities, demand


def calculate_outgoing(array,day,node):
    return sum(array[day][node])


def calculate_incoming(array,day,node):
    total_outgoing = 0
    for n in range(num_nodes):
        total_outgoing = total_outgoing + array[day][n][node]
    return total_outgoing


def demand_day_node_raw(node_capacity, demand_day, left_day, received_day, demand_previous_days, left_previous_days, received_previous_days):
    return demand_day - left_day + received_day+ min(demand_previous_days-left_previous_days+received_previous_days, node_capacity)


def demand_day_node(movements,day,node):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    total_received_patients = 0
    total_outbound_patients = 0
    accumulated_demand = 0
    for d in range(day):
        total_received_patients = total_received_patients + calculate_incoming(movements_res,d,node)
        total_outbound_patients = total_outbound_patients + calculate_outgoing(movements_res,d,node)
    accumulated_demand = sum(demands[node][:day])
    
    demand = max(0, demand_day_node_raw(node_capacities[node],demands[node][day], 
                                        calculate_outgoing(movements_res,day,node), calculate_incoming(movements_res,day,node),
                                        accumulated_demand, total_outbound_patients, total_received_patients                
                ))
    return demand


def calc_total_deaths(movements):
    total = 0
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    for node in range(num_nodes):
        for day in range(num_days):
            deaths_not_attended = max(0, demand_day_node(movements,day,node) - node_capacities[node])
            deaths_transport = prob_death_transport*movements_res[day].sum()
            total = total + deaths_not_attended + deaths_transport
    return total


def outgoing_list(movements):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    outgoing_list = list()
    for day in range(num_days):
        for node in range(num_nodes):
            outgoing_list.append(calculate_outgoing(movements_res,day,node))
    return outgoing_list

def generate_bounds():
    outgoing_list = list()
    for day in range(num_days):
        for outgoing_node in range(num_nodes):
            for incoming_node in range(num_nodes):
                outgoing_list.append((0,transport_capacities[outgoing_node]))
    return outgoing_list

def f_cons(node_id, day):
    return lambda x: transport_capacities[node_id] - calculate_outgoing(np.reshape(x,(num_days,num_nodes,num_nodes)),day ,node_id)

Exception reporting mode: Plain


In [104]:
prob_death_transport = 0
num_days = 10
num_nodes = 4
node_capacities, transport_capacities, demands = init_simulation(num_nodes, 
                                                                 num_days, 
                                                                 icu_capacities=[30000, 20, 80, 1000],
                                                                 transport_capacities=[50, 60, 70, 80], 
                                                                 demand_min = 60, demand_max=100)
print(f"""Randomly generated
{num_days} days, {num_nodes} hospitals
ICU capacities of hospitals: {node_capacities}
Daily transport capacities: {transport_capacities}, 
Daily simulated demand (from uniform distrution): 
{demands}""")

Randomly generated
10 days, 4 hospitals
ICU capacities of hospitals: [30000, 20, 80, 1000]
Daily transport capacities: [50, 60, 70, 80], 
Daily simulated demand (from uniform distrution): 
[[68, 78, 65, 98, 66, 78, 83, 91, 82, 67], [99, 94, 85, 77, 85, 73, 83, 66, 69, 77], [68, 71, 60, 91, 83, 81, 61, 81, 75, 78], [72, 76, 87, 64, 91, 64, 72, 88, 74, 71]]


In [111]:
def split_n(num, num_nodes):
    #TODO this function is not truly random
    pieces = []
    for idx in range(num_nodes-1):
        pieces.append(randint(1,num-sum(pieces)-num_nodes+idx))

    pieces.append(num-sum(pieces))
    return pieces

In [112]:
#define constraints
cons = []
for node in range(num_nodes):
    for day in range(num_days):
        cons.append({'type': 'ineq', 'fun': f_cons(node,day)})

        
#no movement matrix
no_movement = np.array([[[0]*num_nodes]*num_nodes]*num_days)

In [170]:
def random_init(random_days = False, movement_intensity = 1):
    #almost everyone moves everywhere
    random_movement = no_movement.copy()
    for day in range(num_days):
        for node_x in range(num_nodes):
            split_nodes = split_n(np.ceil(transport_capacities[node_x]*movement_intensity), num_nodes)
            for node_y in range(num_nodes):
                if node_x != node_y: #the ones that move to "self" are actually staying
                    random_movement[day, node_x, node_y] += split_nodes[node_y]
                    
    return random_movement

max_movement = random_init()
half_movement = random_init(movement_intensity=0.5)
one_ten_movement = random_init(movement_intensity=0.1)

In [171]:
one_ten_movement.ravel()

array([0, 1, 1, 2, 1, 0, 2, 2, 1, 1, 0, 2, 1, 4, 1, 0, 0, 1, 1, 2, 1, 0,
       1, 3, 1, 2, 0, 2, 4, 1, 1, 0, 0, 1, 1, 2, 1, 0, 1, 3, 1, 2, 0, 2,
       3, 1, 2, 0, 0, 1, 1, 2, 2, 0, 1, 2, 2, 1, 0, 3, 3, 2, 1, 0, 0, 1,
       1, 2, 1, 0, 2, 2, 1, 2, 0, 3, 1, 3, 2, 0, 0, 1, 1, 2, 2, 0, 1, 2,
       3, 1, 0, 2, 4, 1, 1, 0, 0, 1, 1, 2, 1, 0, 1, 3, 1, 2, 0, 2, 2, 1,
       2, 0, 0, 1, 1, 2, 2, 0, 1, 2, 1, 2, 0, 2, 4, 1, 1, 0, 0, 1, 1, 2,
       1, 0, 1, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 1, 1, 2, 1, 0, 1, 3, 3, 1,
       0, 2, 4, 1, 1, 0])

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [196]:
%%time

#all the movements configuration and flatten them with ravel()
movements = [no_movement, 
             random_init(), 
             random_init(movement_intensity=0.75), 
             random_init(movement_intensity=0.5), 
             random_init(movement_intensity=0.25), 
             random_init(movement_intensity=0.1)]
#movements = [x.ravel() for x in movements]

#parameters to minimize
min_deaths = sys.maxsize
best_result = None
result_matrix = None
for movement_intensity in np.arange(0, 1.1, 0.1):
    print(f"Intensity:{movement_intensity:.1f}...", end="")
    movement = random_init(movement_intensity=movement_intensity) if movement_intensity else no_movement
    movement = movement.ravel()
    bounds_movement = generate_bounds()
    #the actual
    result = optimize.minimize(calc_total_deaths, 
                               movement, 
                               method='SLSQP', 
                               constraints=cons, 
                               bounds=bounds_movement)
    
    current_deaths = calc_total_deaths(np.rint(result.x))
    if min_deaths > current_deaths: 
        print("best! ", end="")
        min_deaths = int(current_deaths)
        best_result = result
        result_matrix = np.reshape(np.rint(result.x),(num_days,num_nodes,num_nodes))
print("Done")

Intensity:0.00...best! Intensity:0.10...best! Intensity:0.20...Intensity:0.30...best! Intensity:0.40...Intensity:0.50...Intensity:0.60...Intensity:0.70...Intensity:0.80...best! Intensity:0.90...Intensity:1.00...Done
CPU times: user 46.3 s, sys: 127 ms, total: 46.5 s
Wall time: 46.2 s


### These are the deaths that we get with inaction

In [199]:
calc_total_deaths(no_movement)

1457

### This is the best result found

In [200]:
min_deaths

186

### This the movement matrix

In [201]:
result_matrix

array([[[ 0.,  0.,  0.,  5.],
        [30.,  0.,  0., 31.],
        [52.,  0.,  0., 18.],
        [38.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  5.],
        [44.,  0.,  0., 16.],
        [47.,  0.,  0., 22.],
        [38.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  2.],
        [49.,  0.,  0., 12.],
        [47.,  0.,  0., 23.],
        [17.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  5.],
        [33.,  0.,  0., 26.],
        [54.,  0.,  0., 17.],
        [52.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., 13.],
        [24.,  0., 11., 25.],
        [33.,  0.,  0., 25.],
        [47.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  3.],
        [46.,  0.,  0., 14.],
        [46.,  0.,  0., 24.],
        [41.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  3.],
        [22.,  0.,  0., 38.],
        [57.,  0.,  0., 14.],
        [18.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., 13.],
        [32.,  0.,  0., 29.],
        [23.,  0.,  0., 43.],
        [48.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., 

In [7]:
for node in range(num_nodes):
    for day in range(num_days):
        print(f'Node: {node}, Day: {day}, Demand: {demand_day_node(np.rint(result.x),day,node)}')

Node: 0, Day: 0, Demand: 193.0
Node: 0, Day: 1, Demand: 380.0
Node: 0, Day: 2, Demand: 553.0
Node: 0, Day: 3, Demand: 734.0
Node: 0, Day: 4, Demand: 940.0
Node: 0, Day: 5, Demand: 1136.0
Node: 0, Day: 6, Demand: 1317.0
Node: 0, Day: 7, Demand: 1513.0
Node: 0, Day: 8, Demand: 1690.0
Node: 0, Day: 9, Demand: 1869.0
Node: 1, Day: 0, Demand: 33.0
Node: 1, Day: 1, Demand: 65.0
Node: 1, Day: 2, Demand: 60.0
Node: 1, Day: 3, Demand: 35.0
Node: 1, Day: 4, Demand: 51.0
Node: 1, Day: 5, Demand: 41.0
Node: 1, Day: 6, Demand: 42.0
Node: 1, Day: 7, Demand: 39.0
Node: 1, Day: 8, Demand: 37.0
Node: 1, Day: 9, Demand: 56.0
Node: 2, Day: 0, Demand: 33.0
Node: 2, Day: 1, Demand: 52.0
Node: 2, Day: 2, Demand: 87.0
Node: 2, Day: 3, Demand: 108.0
Node: 2, Day: 4, Demand: 124.0
Node: 2, Day: 5, Demand: 101.0
Node: 2, Day: 6, Demand: 127.0
Node: 2, Day: 7, Demand: 123.0
Node: 2, Day: 8, Demand: 93.0
Node: 2, Day: 9, Demand: 92.0
Node: 3, Day: 0, Demand: 67.0
Node: 3, Day: 1, Demand: 139.0
Node: 3, Day: 2, De

In [150]:
demand_day_node(np.rint(result.x),3,2)

17.0

# SANDBOX

In [53]:
prob_death_transport = 0
num_days = 1
num_nodes = 2
node_capacities, transport_capacities, demands, movements_orig = init_simulation(num_nodes, num_days)
print(f"""Randomly generated
{num_days} days, {num_nodes} hospitals
ICU capacities of hospitals: {node_capacities}
Daily transport capacities: {transport_capacities}, 
Daily simulated demand (from uniform distrution): 
{demands}""")

movements = movements_orig.ravel()

Randomly generated
1 days, 2 hospitals
ICU capacities of hospitals: [277, 154]
Daily transport capacities: [20, 22], 
Daily simulated demand (from uniform distrution): 
[[81], [81]]
